In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [34]:
# ================================================================
#  LEGAL CASE SUMMARIZER — FULL WORKING VERSION (JSON INPUT ONLY)
# ================================================================

from typing import List, Optional, Dict, Any, Tuple, Union
from pydantic import BaseModel, Field


# =======================
#   SCHEMA DEFINITIONS
# =======================

class Statute(BaseModel):
    code: str
    title: Optional[str] = None
    cited_text: Optional[str] = None
    confidence: Optional[float] = None


class SentenceItem(BaseModel):
    text: str
    span: Optional[List[int]] = None
    confidence: Optional[float] = None


class Relation(BaseModel):
    head: str
    relation: str
    tail: str
    evidence_spans: Optional[List[List[int]]] = None
    confidence: Optional[float] = None


class Parties(BaseModel):
    appellant: Optional[str] = None
    respondent: Optional[str] = None


class Metadata(BaseModel):
    label: Optional[str] = None
    court: Optional[str] = None
    bench: Optional[List[str]] = None
    decision_date: Optional[str] = None
    citation: Optional[str] = None
    case_no: Optional[str] = None
    parties: Optional[Parties] = None
    source_file: Optional[str] = None


class Arguments(BaseModel):
    Appellant: List[SentenceItem] = Field(default_factory=list)
    Respondent: List[SentenceItem] = Field(default_factory=list)
    Other: List[SentenceItem] = Field(default_factory=list)


class EnsembleCase(BaseModel):
    case_text: str
    statutes: List[Statute] = Field(default_factory=list)
    facts: List[SentenceItem] = Field(default_factory=list)
    issues: List[SentenceItem] = Field(default_factory=list)
    arguments: Arguments = Field(default_factory=Arguments)
    reasoning: List[SentenceItem] = Field(default_factory=list)
    order: List[SentenceItem] = Field(default_factory=list)
    relations: List[Relation] = Field(default_factory=list)
    metadata: Metadata = Field(default_factory=Metadata)


# =======================
#   VALIDATION HELPERS
# =======================

def _validate(model_cls, data):
    """Handles Pydantic v1/v2 safely."""
    if hasattr(model_cls, "model_validate"):
        return model_cls.model_validate(data)   # v2
    return model_cls.parse_obj(data)            # v1


def _fmt_statute(s: Statute) -> str:
    return f"{s.code} ({s.title})" if s.title else s.code


# ===============================
#   JSON → RUN_CASE ARGUMENTS
# ===============================

def to_run_case_args(ens: Union[EnsembleCase, Dict[str, Any]]):
    """Convert raw JSON → flattened arguments for run_case()."""

    if not isinstance(ens, EnsembleCase):
        ens = _validate(EnsembleCase, ens)

    case_text = ens.case_text.strip()

    statutes = [_fmt_statute(s) for s in ens.statutes]
    facts_sents = [x.text for x in ens.facts]
    issues_sents = [x.text for x in ens.issues]

    arguments_sents = []
    for party, items in [
        ("Appellant", ens.arguments.Appellant),
        ("Respondent", ens.arguments.Respondent),
        ("Other", ens.arguments.Other)
    ]:
        for it in items:
            arguments_sents.append(f"{party}: {it.text}")

    reasoning_sents = [x.text for x in ens.reasoning]
    order_sents = [x.text for x in ens.order]

    label = ens.metadata.label

    return (
        case_text, statutes,
        facts_sents, issues_sents,
        arguments_sents, reasoning_sents,
        order_sents, label
    )


# ===============================
#      SUMMARY GENERATOR
# ===============================

def _make_summary(case_text, statutes, facts_sents, issues_sents,
                  arguments_sents, reasoning_sents, order_sents):

    facts = " ".join(facts_sents)
    issue = issues_sents[0] if issues_sents else ""

    appellant = next((a.split(":",1)[1].strip()
                      for a in arguments_sents if a.startswith("Appellant:")), "")
    respondent = next((a.split(":",1)[1].strip()
                       for a in arguments_sents if a.startswith("Respondent:")), "")

    reasoning = " ".join(reasoning_sents)
    order = " ".join(order_sents)

    # ---------- TECHNICAL STRUCTURED SUMMARY ----------
    technical_summary = {
        "Court": "Supreme Court of India",
        "Statutes": statutes,
        "Facts": facts,
        "Issue": issue,
        "Arguments": {
            "Appellant": appellant,
            "Respondent": respondent
        },
        "Reasoning": reasoning,
        "Order": order
    }

    # ---------- LAYMAN SUMMARY (PARAGRAPH) ----------
    lay = []

    if facts:
        lay.append(f"The case started when {facts.lower()}.")
    if issue:
        lay.append(f"The main question before the court was whether {issue.lower()}.")
    if appellant:
        lay.append(f"The accused argued that {appellant.lower()}.")
    if respondent:
        lay.append(f"The complainant responded that {respondent.lower()}.")
    if reasoning:
        lay.append(f"The court reviewed the evidence and concluded that {reasoning.lower()}.")
    if order:
        lay.append(f"Finally, the court ruled that {order.lower()}.")

    layman_summary = " ".join(lay)

    # ---------- OPTIONAL ONE-LINER ----------
    legal_reasoning_one_liner = (
        "Under Section 420 IPC, the law requires fraudulent intent at the beginning of the transaction."
        if any("420" in s for s in statutes)
        else ""
    )

    return {
        "technical_summary": technical_summary,
        "layman_summary": layman_summary,
        "legal_reasoning_one_liner": legal_reasoning_one_liner
    }


# ===============================
#        FINAL RUN CASE
# ===============================

def run_case(case_text, statutes, facts_sents, issues_sents,
             arguments_sents, reasoning_sents, order_sents, label=None):

    summary = _make_summary(
        case_text, statutes,
        facts_sents, issues_sents,
        arguments_sents, reasoning_sents,
        order_sents
    )

    return {
        "ok": True,
        "label": label,
        "summary": summary
    }


# ===============================
#   WRAPPER (JSON ENTRY POINT)
# ===============================

def run_case_with_ensemble(run_case_fn, ens_json):
    args = to_run_case_args(ens_json)
    return run_case_fn(*args)


In [35]:
def _make_summary(case_text, statutes, facts_sents, issues_sents, arguments_sents, reasoning_sents, order_sents):

    # ---------- Helpers ----------
    def sent(s):
        s = s.strip()
        return s if s.endswith(".") else s + "."

    def clean(s):
        return sent(s.replace("  ", " ").strip())

    def strip_whether(s):
        s = s.strip()
        if s.lower().startswith("whether "):
            return s[8:].rstrip(". ")
        return s.rstrip(". ")

    def lc_first(s):
        return s[0].lower() + s[1:] if s else s

    # ---------- Extract ----------
    facts = [clean(f) for f in facts_sents]
    issue_raw = issues_sents[0] if issues_sents else ""
    issue_core = strip_whether(issue_raw)
    issue_clean = clean(issue_raw)

    appellant_arg = next((a.split(":",1)[1].strip() for a in arguments_sents if a.startswith("Appellant:")), "")
    respondent_arg = next((a.split(":",1)[1].strip() for a in arguments_sents if a.startswith("Respondent:")), "")

    appellant_arg = clean(appellant_arg) if appellant_arg else ""
    respondent_arg = clean(respondent_arg) if respondent_arg else ""

    reasoning = [clean(r) for r in reasoning_sents]
    reasoning_text = " ".join(reasoning)
    order_text = clean(" ".join(order_sents)) if order_sents else ""

    # =====================================================
    #      TECHNICAL SUMMARY (JUDICIAL, FORMAL, LEGAL)
    # =====================================================
    tech = []

    if facts:
        tech.append(
            f"The present appeal before this Court arises from allegations wherein {lc_first(facts[0])}"
        )
        if len(facts) > 1:
            tech.append(" ".join(facts[1:]))

    if issue_core:
        tech.append(
            f"The central question requiring determination is whether {lc_first(issue_core)}."
        )

    if appellant_arg:
        tech.append(
            f"Learned counsel for the appellant submitted that {lc_first(appellant_arg)}"
        )
    if respondent_arg:
        tech.append(
            f"Conversely, learned counsel for the respondent contended that {lc_first(respondent_arg)}"
        )

    if reasoning_text:
        tech.append(
            f"Upon an evaluation of the evidentiary record and the statutory ingredients, the Court was of the view that {lc_first(reasoning_text)}"
        )

    if order_text:
        tech.append(
            f"In view of the foregoing analysis, the Court held that {lc_first(order_text)}"
        )

    technical_paragraph = " ".join(tech)

    # =====================================================
    #      LAYMAN SUMMARY (VERY SIMPLE, STORY FORMAT)
    # =====================================================
    lay = []

    # Story-like framing, different style
    if facts:
        lay.append(
            f"The problem started when {lc_first(facts[0]).replace(';', ',')}"
        )
        if len(facts) > 1:
            lay.append(" ".join(facts[1:]))

    if issue_core:
        lay.append(
            f"The court basically had to figure out whether {lc_first(issue_core)}."
        )

    if appellant_arg:
        lay.append(
            f"The accused person argued that {lc_first(appellant_arg)}"
        )

    if respondent_arg:
        lay.append(
            f"On the other hand, the complainant said that {lc_first(respondent_arg)}"
        )

    if reasoning_text:
        lay.append(
            f"After looking at everything, the judges felt that {lc_first(reasoning_text)}"
        )

    if order_text:
        lay.append(
            f"In simple terms, the court finally decided that {lc_first(order_text)}"
        )

    layman_summary = " ".join(lay)

    # ------------------------------------------------------
    # Legal One-Liner
    # ------------------------------------------------------
    legal_one_liner = (
        "Under Section 420 IPC, the prosecution must prove that the accused acted with dishonest intent at the very outset of the transaction."
        if any("420" in s for s in statutes)
        else ""
    )

    # Structured metadata for technical view
    technical_struct = {
        "Court": "Supreme Court of India",
        "Statutes": statutes,
        "Facts": " ".join(facts),
        "Issue": issue_clean,
        "Arguments": {"Appellant": appellant_arg, "Respondent": respondent_arg},
        "Reasoning": reasoning_text,
        "Order": order_text
    }

    return {
        "technical_summary": technical_struct,
        "technical_paragraph": technical_paragraph,
        "layman_summary": layman_summary,
        "legal_reasoning_one_liner": legal_one_liner,
    }


In [36]:
# Cell 3: Test input JSON

test_input = {
    "case_text": "This case concerns allegations of cheating relating to a property transaction.",

    "statutes": [
        {"code": "Section 420 IPC", "title": "Cheating"},
        {"code": "Section 415 IPC", "title": "Definition of Cheating"}
    ],

    "facts": [
        {"text": "The complainant paid ₹12,00,000 after the accused claimed he had rights over the property."},
        {"text": "No registered sale deed was executed despite repeated follow-ups."}
    ],

    "issues": [
        {"text": "Whether the accused had dishonest intention at the beginning of the transaction."}
    ],

    "arguments": {
        "Appellant": [
            {"text": "The dispute is purely civil in nature; there was no intention to deceive."}
        ],
        "Respondent": [
            {"text": "The accused knew from the beginning that he had no title to the property."}
        ],
        "Other": []
    },

    "reasoning": [
        {"text": "Dishonest intent at the inception is the core requirement under Section 420 IPC."},
        {"text": "Evidence showed the accused had no legal right over the property."}
    ],

    "order": [
        {"text": "The appeal is dismissed. Conviction under Section 420 IPC is upheld."}
    ],

    "metadata": {"label": "Test Case – IPC 420"}
}


In [37]:
# Cell 4: Run summarizer with clean paragraph formatting

output = run_case_with_ensemble(run_case, test_input)

# Full dictionary if needed
# output

# Pretty printing helper
def pretty_print(text):
    import textwrap
    wrapped = textwrap.fill(text, width=90)  # wrap at 90 chars per line
    print(wrapped)
    print("\n" + "-"*90 + "\n")

print("\n================ TECHNICAL PARAGRAPH ================\n")
pretty_print(output["summary"]["technical_paragraph"])

print("\n================ LAYMAN SUMMARY ================\n")
pretty_print(output["summary"]["layman_summary"])

print("\n================ LEGAL ONE-LINER ================\n")
pretty_print(output["summary"]["legal_reasoning_one_liner"])



================ TECHNICAL PARAGRAPH ================

The present appeal before this Court arises from allegations wherein the complainant paid
₹12,00,000 after the accused claimed he had rights over the property. No registered sale
deed was executed despite repeated follow-ups. The central question requiring
determination is whether the accused had dishonest intention at the beginning of the
transaction. Learned counsel for the appellant submitted that the dispute is purely civil
in nature; there was no intention to deceive. Conversely, learned counsel for the
respondent contended that the accused knew from the beginning that he had no title to the
property. Upon an evaluation of the evidentiary record and the statutory ingredients, the
Court was of the view that dishonest intent at the inception is the core requirement under
Section 420 IPC. Evidence showed the accused had no legal right over the property. In view
of the foregoing analysis, the Court held that the appeal is dismiss